In [3]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd

In [ ]:
#For Specialization
spl_title=[]
spl_link=[]


for i in range(1,16):
    #44&index=prod_all_products_term_optimization_v3&entityTypeDescription=Specializations&allLanguages=English
    url = "https://www.coursera.org/directory/specializations?page=" +str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    st = soup.find_all('a', class_ ="c-directory-link")
    num=len(st)
    for j in range(1,num):
        #spl-title
        st = soup.find_all('a', class_ ="c-directory-link")[j].get_text()
        spl_title.append(st)       
        #spl URL
        curl= soup.find_all("a", class_='c-directory-link')[j]
        spl_link.append("https://www.coursera.org"+ curl.get('href'))

#creating a dataframe of all specializations
splz=pd.DataFrame({'spl_title':spl_title,
                   'spl_link':spl_link})

#writing into a .csv file
#coursera_df.to_csv('Coursera_catalog.csv')

In [2]:
#get all courses on coursera
course_title=[]
course_link=[]

cnt=0
for i in range(1,153):
    #44&index=prod_all_products_term_optimization_v3&entityTypeDescription=Specializations&allLanguages=English
    url = "https://www.coursera.org/directory/courses?page=" +str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    st = soup.find_all('a', class_ ="c-directory-link")
    num=len(st)
    for j in range(1,num):
        #course-title
        ct = soup.find_all('a', class_ ="c-directory-link")[j].get_text()
        cnt=cnt+1
        course_title.append(ct)       
        #course URL
        curl= soup.find_all("a", class_='c-directory-link')[j]
        course_link.append("https://www.coursera.org"+ curl.get('href'))

#creating a dataframe of all specializations
courses=pd.DataFrame({'course_title':course_title,
                   'course_link':course_link})

#writing into a .csv file
#coursera_df.to_csv('Coursera_catalog.csv')

In [5]:
courses.course_title.at[25]

'System Administration and IT Infrastructure Services (Google)'

In [6]:
cs=courses.iloc[[16,27,24,25]]
uuur=cs.course_link

In [7]:
cs=courses.iloc[[27,24]]
uuur=cs.course_link

In [ ]:
uuur

27    https://www.coursera.org/learn/use-canva-to-cr...
24    https://www.coursera.org/learn/supervised-lear...
Name: course_link, dtype: object

In [15]:
#To get detail of each course

course_name=[]
course_type=[]
course_photo_URL=[]
typ=[]
typ_name=[]
university_name=[]
university_logo=[]
spl_skills=[]
course_tit=[]
time_required=[]
level=[]


for urlC in uuur:
    page_course = requests.get(urlC)
    soup = BeautifulSoup(page_course.content, 'html.parser')
    
    guided=soup.find('div',class_="_fc5ifbq _bv93ce")
    if guided is None:
      #Attaching course Name/Title
      cnn=soup.find('h1',class_="banner-title m-b-0 banner-title-without--subtitle").get_text()
      
      #Attaching type of course
      typ_name.append('COURSE')
      
      #Attaching course level(Beginner/Intermediate/ Advanced) and  time required to complete
      fet=soup.find_all('div',class_='_16ni8zai m-b-0 m-t-1s')
      for i in range(0,2):

        if i==0:
          lev=fet[i].get_text()
          level.append(lev)
        else:
          vtim=fet[i].get_text()
          time_required.append(vtim)
      
      
      #Attaching Course Languages
      lang=soup.find_all('div',class_="_16ni8zai m-b-0")
      lan=lang[3].get_text()
      print(lan)

        


        
        




English


In [41]:
#Attaching Course Languages
u="https://www.coursera.org/learn/machine-learning-projects"
page_course = requests.get(u)
soup = BeautifulSoup(page_course.content, 'html.parser')
lang=soup.find_all(id="Availablelanguages7a3347e3-ac02-4828-93a0-1470665e5bc2")
lang

[]

In [40]:
lang

In [ ]:
course_skills=[]
skills = []
course_top_instructor=[]
sub_course_list=[]
course_list=[]

In [ ]:
typ=[]
typ_name=[]
university_logo=[]
spl_skills=[]
course_tit=[]

# getting course info
for urlC in course_spl.course_link:
    page_course = requests.get(urlC)
    soup_course = BeautifulSoup(page_course.content, 'html.parser')
    # to get specialization skills
    skills= [x.get_text() for x in soup_course.find_all('span', class_ = '_rsc0bd m-r-1s m-b-1s')]
    spl_skills.append(skills)
    skills=[]
    cl= soup_course.find_all('div', class_ = '_jyhj5r CourseItem')
    for c in cl:
        
        course_title=c.find('h3',class_="headline-3-text bold m-t-1 m-b-2").get_text()
        
        course_tit.append(course_title)
    

In [ ]:
print(course_skills[:5])
print(len(course_skills))

[['Debugging', 'Encryption Algorithms and Techniques', 'Customer Service', 'Network Protocols', 'Cloud Computing', 'Binary Code', 'Customer Support', 'Linux', 'Troubleshooting', 'Domain Name System (DNS)', 'Ipv4', 'Network Model'], ['Data Science', 'Statistical Analysis', 'Machine Learning', 'Python Programming', 'Business Intelligence', 'Data Analysis', 'Pandas', 'Numpy', 'Cloud Databases', 'Ipython', 'Relational Database Management System (RDBMS)', 'SQL'], ['Data Science', 'Relational Database Management System (RDBMS)', 'Cloud Databases', 'Python Programming', 'SQL', 'Ipython'], ['Tensorflow', 'Convolutional Neural Network', 'Artificial Neural Network', 'Deep Learning', 'Backpropagation', 'Python Programming', 'Hyperparameter', 'Hyperparameter Optimization', 'Machine Learning', 'Inductive Transfer', 'Multi-Task Learning', 'Facial Recognition System'], ['Json', 'Xml', 'Python Programming', 'Database (DBMS)', 'Python Syntax And Semantics', 'Basic Programming Language', 'Computer Progr

In [ ]:
coursera_df['course_skills']=course_skills
coursera_df['course_top_instructor']=course_top_instructor
coursera_df['sub_course_list']=sub_course_list

In [ ]:
print(coursera_df)

In [ ]:
coursera_df.sub_course_list.to_excel('output1.xlsx')

In [ ]:
coursera_df.to_csv('Coursera_catalog.csv')


In [ ]:
#Getting Instructors
for urlC in course_URL:
    url_course="https://www.coursera.org/specializations/deep-learning"
    page_course = requests.get(url_course)
    soup_course = BeautifulSoup(page_course.content, 'html.parser')
    instructor= soup_course.find('h3', class_ = 'instructor-name headline-3-text bold').get_text()
    course_top_instructor.append(instructor[:-14])
    
    cl= soup_course.find_all('div', class_ = '_jyhj5r CourseItem')
    for c in cl:
        course_number=c.find('span',class_="_1nc68rjl text-secondary d-block m-y-1").get_text()
        course_title=c.find('h3',class_="headline-3-text bold m-t-1 m-b-2").get_text()
        course_list.append(course_number.strip())
        course_list.append(course_title)
    sub_course_list.append(course_list)


KeyboardInterrupt: ignored

In [ ]:
# Getting Course List and details
cl= soup_course.find_all('div', class_ = '_jyhj5r CourseItem')
for c in cl:
    course_number=c.find('span',class_="_1nc68rjl text-secondary d-block m-y-1").get_text()
    course_title=c.find('h3',class_="headline-3-text bold m-t-1 m-b-2").get_text()
    course_list.append(course_number)
    course_list.append(course_title)
sub_course_list.append(course_list)

In [ ]:
# Getting course price
url_price="https://www.coursera.org/professional-certificates/google-it-support#enrol"
page_price = requests.get(url_price)
soup_price = BeautifulSoup(page_price.content, 'html.parser')
price= [x.get_text() for x in soup_price.find_all('span')]
print(price)
# since frice is in a modal-container, there are difficulties faced in scraping it